In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import random

# --- ENVIRONMENT SETUP ---
np.random.seed(2)
n_states = 3
n_actions = 3

# Transition probability matrices
P = np.zeros((n_actions, n_states, n_states))
for a in range(n_actions):
    for s in range(n_states):
        row = np.random.rand(n_states)
        P[a][s] = row / row.sum()

# Reward matrix
R = np.random.uniform(0, 2, size=(n_states, n_actions))

# --- (1) GENERATE EXPERIENCES ---
def generate_experiences(P, R, n_states, n_actions, num_episodes=500, max_steps=10):
    experiences = []
    for _ in range(num_episodes):
        s = np.random.randint(0, n_states)
        for _ in range(max_steps):
            a = np.random.randint(0, n_actions)
            s_prime = np.random.choice(n_states, p=P[a][s])
            r = R[s, a]
            experiences.append((s, a, r, s_prime))
            s = s_prime
    return experiences

experiences = generate_experiences(P, R, n_states, n_actions)




# --- (2) DEFINE DQN ---
class DQN(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(DQN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(state_dim, 32),
            nn.ReLU(),
            nn.Linear(32, action_dim)
        )

    def forward(self, x):
        return self.net(x)

def one_hot(s, n_states):
    vec = np.zeros(n_states)
    vec[s] = 1.0
    return vec

# --- HYPERPARAMETERS ---
gamma = 0.95
alpha = 0.001
batch_size = 32
epochs = 100

# Initialize networks
policy_net = DQN(n_states, n_actions)
target_net = DQN(n_states, n_actions)
target_net.load_state_dict(policy_net.state_dict())
optimizer = optim.Adam(policy_net.parameters(), lr=alpha)
loss_fn = nn.MSELoss()

# Replay buffer
memory = deque(experiences, maxlen=10000)

# --- TRAINING LOOP ---
for epoch in range(epochs):
    batch = random.sample(memory, batch_size)
    states, actions, rewards, next_states = zip(*batch)

    states_tensor = torch.tensor([one_hot(s, n_states) for s in states], dtype=torch.float32)
    next_states_tensor = torch.tensor([one_hot(s_, n_states) for s_ in next_states], dtype=torch.float32)
    actions_tensor = torch.tensor(actions, dtype=torch.long).unsqueeze(1)
    rewards_tensor = torch.tensor(rewards, dtype=torch.float32).unsqueeze(1)

    q_values = policy_net(states_tensor).gather(1, actions_tensor)
    next_q_values = target_net(next_states_tensor).max(1, keepdim=True)[0].detach()
    target = rewards_tensor + gamma * next_q_values

    loss = loss_fn(q_values, target)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        target_net.load_state_dict(policy_net.state_dict())

# --- FINAL Q-VALUES ---
final_q_values = policy_net(torch.eye(n_states)).detach().numpy()
print("Final learned Q-values (Deep Q-Network):")
print(np.round(final_q_values, 3))


Final learned Q-values (Deep Q-Network):
[[1.394 1.306 1.255]
 [0.921 1.02  1.167]
 [1.022 1.088 1.052]]


C:\Users\wilso\AppData\Local\Temp\ipykernel_40864\1155754181.py:77: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:281.)
  states_tensor = torch.tensor([one_hot(s, n_states) for s in states], dtype=torch.float32)


In [2]:
# Verificar Bellman para todos los (s, a)
def check_bellman_equation(Q_star, R, P, gamma=0.95):
    n_states, n_actions = R.shape
    residuals = np.zeros_like(Q_star)

    for s in range(n_states):
        for a in range(n_actions):
            rhs = R[s, a] + gamma * np.dot(P[a][s], np.max(Q_star, axis=1))
            lhs = Q_star[s, a]
            residuals[s, a] = lhs - rhs
            print(f"Q*({s},{a}) = {lhs:.4f} | Bellman RHS = {rhs:.4f} | Residual = {lhs - rhs:.4f}")

    return residuals

# Ejecutar verificación
residuals = check_bellman_equation(final_q_values, R, P, gamma=0.95)


Q*(0,0) = 1.3938 | Bellman RHS = 1.3748 | Residual = 0.0191
Q*(0,1) = 1.3057 | Bellman RHS = 1.5621 | Residual = -0.2564
Q*(0,2) = 1.2554 | Bellman RHS = 1.9094 | Residual = -0.6539
Q*(1,0) = 0.9207 | Bellman RHS = 2.1026 | Residual = -1.1819
Q*(1,1) = 1.0205 | Bellman RHS = 1.5306 | Residual = -0.5102
Q*(1,2) = 1.1674 | Bellman RHS = 2.4014 | Residual = -1.2339
Q*(2,0) = 1.0221 | Bellman RHS = 2.0943 | Residual = -1.0722
Q*(2,1) = 1.0882 | Bellman RHS = 2.1813 | Residual = -1.0930
Q*(2,2) = 1.0519 | Bellman RHS = 1.8937 | Residual = -0.8419


In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import random

# --- ENVIRONMENT SETUP ---
np.random.seed(2)
n_states = 3
n_actions = 3

# Transition probability matrices
P = np.zeros((n_actions, n_states, n_states))
for a in range(n_actions):
    for s in range(n_states):
        row = np.random.rand(n_states)
        P[a][s] = row / row.sum()

# Reward matrix
R = np.random.uniform(0, 2, size=(n_states, n_actions))

# --- EXPERIENCE GENERATION ---
def generate_experiences(P, R, n_states, n_actions, num_episodes=2000, max_steps=10):
    experiences = []
    for _ in range(num_episodes):
        s = np.random.randint(0, n_states)
        for _ in range(max_steps):
            a = np.random.randint(0, n_actions)
            s_prime = np.random.choice(n_states, p=P[a][s])
            r = R[s, a]
            experiences.append((s, a, r, s_prime))
            s = s_prime
    return experiences

experiences = generate_experiences(P, R, n_states, n_actions)

# --- DQN MODEL ---
class DQN(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(DQN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(state_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, action_dim)
        )

    def forward(self, x):
        return self.net(x)

def one_hot(s, n_states):
    vec = np.zeros(n_states)
    vec[s] = 1.0
    return vec

# --- HYPERPARAMETERS ---
gamma = 0.95
alpha = 0.005
batch_size = 64
epochs = 1000

# Initialize networks
policy_net = DQN(n_states, n_actions)
target_net = DQN(n_states, n_actions)
target_net.load_state_dict(policy_net.state_dict())
optimizer = optim.Adam(policy_net.parameters(), lr=alpha)
loss_fn = nn.MSELoss()

# Replay buffer
memory = deque(experiences, maxlen=10000)

# --- TRAINING LOOP ---
for epoch in range(epochs):
    batch = random.sample(memory, batch_size)
    states, actions, rewards, next_states = zip(*batch)

    states_tensor = torch.tensor([one_hot(s, n_states) for s in states], dtype=torch.float32)
    next_states_tensor = torch.tensor([one_hot(s_, n_states) for s_ in next_states], dtype=torch.float32)
    actions_tensor = torch.tensor(actions, dtype=torch.long).unsqueeze(1)
    rewards_tensor = torch.tensor(rewards, dtype=torch.float32).unsqueeze(1)

    q_values = policy_net(states_tensor).gather(1, actions_tensor)
    next_q_values = target_net(next_states_tensor).max(1, keepdim=True)[0].detach()
    target = rewards_tensor + gamma * next_q_values

    loss = loss_fn(q_values, target)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        target_net.load_state_dict(policy_net.state_dict())

# --- FINAL Q* ---
Q_star = policy_net(torch.eye(n_states)).detach().numpy()
print("Final Q* (approx):\n", np.round(Q_star, 3))

# --- BELLAMAN VERIFICATION ---
def check_bellman_equation(Q_star, R, P, gamma=0.95):
    print("\nVerifying Bellman Equation:")
    n_states, n_actions = R.shape
    residuals = np.zeros_like(Q_star)
    for s in range(n_states):
        for a in range(n_actions):
            rhs = R[s, a] + gamma * np.dot(P[a][s], np.max(Q_star, axis=1))
            lhs = Q_star[s, a]
            residuals[s, a] = lhs - rhs
            print(f"Q*({s},{a}) = {lhs:.4f} | Bellman RHS = {rhs:.4f} | Residual = {lhs - rhs:.4f} as % of Q* = {(lhs - rhs) / lhs * 100:.2f}%")
    return residuals

residuals = check_bellman_equation(Q_star, R, P, gamma=0.95)


Final Q* (approx):
 [[20.08  20.8   20.681]
 [21.01  20.907 21.816]
 [21.205 21.174 21.205]]

Verifying Bellman Equation:
Q*(0,0) = 20.0796 | Bellman RHS = 20.2077 | Residual = -0.1281 as % of Q* = -0.64%
Q*(0,1) = 20.7999 | Bellman RHS = 20.7673 | Residual = 0.0326 as % of Q* = 0.16%
Q*(0,2) = 20.6809 | Bellman RHS = 20.6492 | Residual = 0.0317 as % of Q* = 0.15%
Q*(1,0) = 21.0101 | Bellman RHS = 21.1448 | Residual = -0.1347 as % of Q* = -0.64%
Q*(1,1) = 20.9073 | Bellman RHS = 20.8440 | Residual = 0.0633 as % of Q* = 0.30%
Q*(1,2) = 21.8158 | Bellman RHS = 21.8041 | Residual = 0.0117 as % of Q* = 0.05%
Q*(2,0) = 21.2047 | Bellman RHS = 21.3607 | Residual = -0.1559 as % of Q* = -0.74%
Q*(2,1) = 21.1740 | Bellman RHS = 21.2459 | Residual = -0.0718 as % of Q* = -0.34%
Q*(2,2) = 21.2052 | Bellman RHS = 21.2316 | Residual = -0.0264 as % of Q* = -0.12%
